In [2]:
from SemanticCorruption import *

In [3]:
from nltk.corpus import wordnet as wn
import pattern.en as en
import numpy as np
import itertools

In [8]:
sents = ["It's not too bad but makes a lot of unnatural sounding sentences.",
'A Washington County man may have the countys first human case of West Nile virus, the health department said Friday',
"It was a dusty, dry, hot day, and the flys were buzzing.",
"The article is the most common determiner (DT) in English.",
"We may have a question",
"Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?",
"Both gerunds and infinitives can be used as the subject or the complement of a sentence.",
"Shares of Xoma fell 16 percent in early trade, while shares of Genentech, a much larger company with several products on the market, were up 2 percent.",
"Six months ago, the IMF and Argentina struck a bare-minimum $6.8-billion debt rollover deal that expires in August.",
"He plans to have dinner with troops at Kosovo's U.S. military headquarters, Camp Bondsteel.",
"After that, he plans to have dinner at Camp Bondsteel with U.S. troops stationed there.",
"He added that prosecutors will seek the death penalty.",
"Who is this man?",
"Who is that man?",
"This evil thief stole that car!",
"The motorist is angry, so the pedestrian is understandably scared",
"The hero was brave, he defeated the dragon.",
"The villian was cowardly, he was slain by the dragon.",
"War and Peace opens in the Russian city of St. Petersburg in 1805, as Napoleon's conquest of western Europe is just beginning to stir fears in Russia.",
"PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer.",
"You sing very well.",
"We look forward to your talk.",
"I knew French.",
         
        ]

In [57]:

for ii,sent in enumerate(sents):
    print(ii)
    print(sent)
    print("\n".join(leveled_semantic_corrupt_noun_synonym_sentences(sent)))
    print("-"*50)

0
It's not too bad but makes a lot of unnatural sounding sentences.
It 's not too bad but makes a lot of unnatural sounding convictions .
--------------------------------------------------
1
A Washington County man may have the countys first human case of West Nile virus, the health department said Friday
a Washington County serviceman may have the countys first human case of West Nile virus , the health department said Friday
a Washington County serviceman may have the counties first human case of West Nile virus , the health department said Friday
a Washington County serviceman may have the counties first human case of West Nile virus , the health section said Friday
a Washington County serviceman may have the counties first human case of West Nile virus , the wellness section said Friday
a Washington County serviceman may have the counties first human slip of West Nile virus , the wellness section said Friday
--------------------------------------------------
2
It was a dusty, dry, 

In [4]:
import sys
sys.path.append("../../../Resources/tools/lib/python/pywsd/")
import pywsd
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim

In [5]:
def make_pywsd_sent(tagged_words):
    return[(word,word,pennPOS2WordnetPOS(tag)) for (word,tag) in tagged_words]

In [8]:
import nltk.wsd


sent = sents[5]
words, tagged_words = tokenize_and_tag(sent)
wsd_sent =make_pywsd_sent(get_tagged_phrases(tagged_words,3))
ans=disambiguate(wsd_sent, context_is_lemmatized=True, nbest=True, normalizescore=True, keepscore=True, algorithm=pywsd.lesk.simple_lesk)
ans

(u'Is', u'Is', None)
(u'changing', u'changing', u'v')
(u'an', u'an', None)
(u'odd', u'odd', u'a')
(u'number', u'number', u'n')
(u'of', u'of', None)
(u'verbs', u'verbs', u'n')
(u',', u',', None)
(u'adverbs', u'adverbs', u'n')
(u',', u',', None)
(u'adjectives', u'adjectives', u'n')
(u'and', u'and', None)
(u'nouns', u'nouns', u'n')
(u'to', u'to', None)
(u'their', u'their', None)
(u'antonyms', u'antonyms', u'n')
(u'expected', u'expected', u'v')
(u'to', u'to', None)
(u'produce', u'produce', u'v')
(u'a', u'a', None)
(u'semantically', u'semantically', u'r')
(u'distant', u'distant', u'a')
(u'sentence', u'sentence', u'n')
(u'?', u'?', None)


In [4]:
# Loading the Wordnet domains.
from collections import defaultdict

from glob import glob
import os.path


domain_files = glob('../../../Resources/tools/lib/databases/xwnd-30g/*.ppv')
synset2domains = defaultdict(lambda: np.zeros(len(domain_files)))
for domain_index,domain_filename in enumerate(domain_files):
    domain = os.path.splitext(os.path.basename(domain_filename))[0]
    with open(domain_filename,'r') as fh:
        for line in list(fh):
            ssid, weight = line.split()
            weight = float(weight)
            #assert not np.isnan(weight)
            
            synset2domains[ssid][domain_index] = weight 


def get_domains(synset):
    return synset2domains[get_synset_id(synset)]

def get_synset_id(synset):
    return str(synset.offset()).zfill(8) + "-" + synset.pos()

In [29]:
np.power([1,2,3],4)

array([ 1, 16, 81])

In [30]:
def get_scaled_domains(synsets):
    def count(synset):
        return sum([1.0*lemma.count() for lemma in synset.lemmas()])
    
    counts = np.asarray([count(ss) for ss in synsets])
    counts+=1.0
    counts/=counts.sum()
    counts = np.power(counts,2)
    domains = [counts[ii]*get_domains(ss) for ii,ss in enumerate(synsets)]
    return domains


In [28]:
synset_misses = set()
def try_hard_to_get_synsets(word, pos):
    synsets = wn.synsets(word,pos)

#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace(".",""),pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace("-","_"),pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace("-",""),pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace("_",""),pos)
#    if len(synsets)==0 and word=='%':
#        synsets = wn.synsets('percent',pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("_")[-1],pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("-")[-1],pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("_")[0],pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("-")[0],pos)
        
#    if len(synsets)==0 and not pos is None:
#        synsets = try_hard_to_get_synsets(word, None)
        
#    if len(synsets)==0:
#        synset_misses.add((word,pos))
    
        
    return synsets

In [31]:
def domain_wsd(tagged_phrases):
    def tag2wn_pos(tag):
        tag = 'a' if tag[0] =='J' else tag
        tag = 'n' if tag =='CD' else tag
        tag = '*' if tag =='VHP' else tag #Blank it
        tag = tag[0].lower()
        return tag if tag in ['a','v','r','n'] else None

    sentence_domain = np.zeros(len(domain_files))

    for lemma,pos in tagged_phrases:
        wn_pos = tag2wn_pos(pos)
        if wn_pos: 
            domain_scores=get_scaled_domains(try_hard_to_get_synsets(lemma, wn_pos))
            for domain_score in domain_scores:
                sentence_domain+=domain_score
    
    def get_best_synsets():
        for lemma,pos in tagged_phrases:
            best_synset = None
            wn_pos = tag2wn_pos(pos)
            if wn_pos:
                best_score = -1*np.Inf
                synsets = try_hard_to_get_synsets(lemma, wn_pos)
                domain_scores=get_scaled_domains(synsets)
                for ii,ss in enumerate(synsets):
                    score = np.dot(domain_scores[ii],sentence_domain)
                    if score>best_score:
                        best_score=score
                        best_synset=ss
            #yield lemma, best_synset, None if best_synset is None else best_synset.definition()
            yield lemma, best_synset
            
    return list(get_best_synsets())

In [6]:
def domain_wsd_on_plaintext(sent, max_phrase_len=3):
    words, tagged_words = tokenize_and_tag(sent)
    tagged_phrases = get_tagged_phrases(tagged_words,max_phrase_len)
    return domain_wsd(tagged_phrases)
    

In [10]:
sent = sents[19]
print sent
print "--------------"
sent_dom = domain_wsd_on_plaintext(sent)
sent_dom

PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer.
--------------


[(u'PCCW', None, None),
 (u"'s", None, None),
 (u'chief_operating_officer',
  Synset('chief_executive_officer.n.01'),
  u'the corporate executive responsible for the operations of the firm; reports to a board of directors; may appoint other managers (including a president)'),
 (u',', None, None),
 (u'Mike',
  Synset('microphone.n.01'),
  u'device for converting sound waves into electrical energy'),
 (u'Butcher',
  Synset('butcher.v.01'),
  u'kill (animals) usually for food consumption'),
 (u',', None, None),
 (u'and', None, None),
 (u'the', None, None),
 (u'Arena',
  Synset('sphere.n.01'),
  u'a particular environment or walk of life'),
 (u'brothers',
  Synset('brother.n.05'),
  u'(Roman Catholic Church) a title given to a monk and used as form of address'),
 (u',', None, None),
 (u'the', None, None),
 (u'chief_financial_officers',
  Synset('chief_financial_officer.n.01'),
  u'the corporate executive having financial authority to make appropriations and authorize expenditures for a fir

In [32]:
def get_sense_key(synset, lemma):
    index = 0
    try: #try and do better
        index = synset.lemma_names().index(lemma)
    except ValueError:
        pass #couldn't do better
    return synset.lemmas()[index].key()

def wsd(text_id, tagged_phrases, indexed_instances):
    lemmas_and_synsets = domain_wsd(tagged_phrases)
    for ii, id in indexed_instances.items():
        #print(id)
        lemma, synset = lemmas_and_synsets[ii]
        comment = "!! lemma="+''.join([i if ord(i) < 128 else ' ' for i in lemma])
        if synset: #If it was even defined in wordnet
            sensekey = get_sense_key(synset,lemma)
            yield text_id, id, sensekey, comment
        else:
            yield text_id, id, comment

In [9]:
import xml.etree.ElementTree as ET
def tagged_sentences(filename):
    xml = ET.ElementTree(file=filename)
    for text in xml.getroot():
        text_id = text.get('id')
        
        for sentence in text:
            tagged_phrases = []
            indexed_instances = dict()
            for ii,phrase in enumerate(sentence):
                lemma = phrase.get('lemma')
                lemma = phrase.text if lemma=="@card@" else lemma  #correct it to use the text if it was a number
                
                tagged_phrases.append((lemma,phrase.get('pos')))
                if 'id' in phrase.keys():
                    indexed_instances[ii] = phrase.get('id')
            yield text_id, tagged_phrases, indexed_instances    
    

    

In [10]:
corpus_filename = "./SemEval13_task12_data/test.en.xml"


In [33]:
answers = list(itertools.chain(*[wsd(*tti) for tti in tagged_sentences(corpus_filename)]))

In [17]:
synset_misses

{('-RRB-', None),
 ('-RRB-', 'n'),
 ('0,6', None),
 ('0,6', 'n'),
 ('0,9', None),
 ('0,9', 'n'),
 ('0.03', None),
 ('0.03', 'n'),
 ('0.37', None),
 ('0.37', 'n'),
 ('0.55', None),
 ('0.55', 'n'),
 ('0.63', None),
 ('0.63', 'n'),
 ('102', None),
 ('102', 'n'),
 ('106', None),
 ('106', 'n'),
 ('106.41', None),
 ('106.41', 'n'),
 ('126', None),
 ('126', 'n'),
 ('127', None),
 ('127', 'n'),
 ('13,000', None),
 ('13,000', 'n'),
 ('14.2', None),
 ('14.2', 'n'),
 ('140,000', None),
 ('140,000', 'n'),
 ('150,000', None),
 ('150,000', 'n'),
 ('190.31', None),
 ('190.31', 'n'),
 ('1966', None),
 ('1966', 'n'),
 ('1976', None),
 ('1976', 'n'),
 ('1990', None),
 ('1990', 'n'),
 ('1995', None),
 ('1995', 'n'),
 ('20.5', None),
 ('20.5', 'n'),
 ('2001', None),
 ('2001', 'n'),
 ('2002', None),
 ('2002', 'n'),
 ('2003', None),
 ('2003', 'n'),
 ('2005', None),
 ('2005', 'n'),
 ('2007', None),
 ('2007', 'n'),
 ('2008', None),
 ('2008', 'n'),
 ('2009', None),
 ('2009', 'n'),
 ('2011', None),
 ('2011', 'n

In [34]:
with open("./SemEval13_task12_data/my_answers_test.txt",'w') as fh:
    for answer_parts in answers:
        fh.write(" ".join(answer_parts)+'\n')
        
        
    

In [80]:
l.key()

u'drug_war%1:04:00::'

In [85]:
wn.synsets("colombia")[0].lemmas()[0].key()

u'colombia%1:15:00::'

In [92]:
for i,j in {"a":1,"b":"2" }.items():
    print (i,j)

('a', 1)
('b', '2')


In [126]:
ss=wn.synsets("month")[0]
ss.lemmas()[1].key()

u'month%1:28:01::'

In [130]:
ss.lemma_names()

AttributeError: 'list' object has no attribute 'find'

In [253]:
#key   = emission%1:27:00::
#guess = emission%1:04:00::/1.000

In [257]:
get_sense_key(wn.synsets("emission")[0], "emission")

u'emission%1:04:00::'

In [258]:
get_sense_key(wn.synsets("emission")[1], "emission")

u'emission%1:27:00::'

In [260]:
wn.synsets("emission")[0].definition()

u'the act of emitting; causing to flow forth'

In [261]:
wn.synsets("emission")[1].definition()

u'a substance that is emitted or released'

In [225]:
wn.synsets("money")[2].definition()

u'the official currency issued by a government or national bank'

In [226]:
wn.synsets("money")[2].lemmas()[0].key()

u'money%1:21:01::'

In [228]:
l.count()

77

In [229]:
wn.synsets("money")[2].lemmas()[0].count()

0

In [265]:
wn.synsets("UN")

[Synset('united_nations.n.01')]